In [ ]:
# install peft and torch and datasets
!pip install requests transformers

In [ ]:
import torch

# 1. Load the translation dataset ("opus_books", "en-fr")
dataset = None


In [ ]:

# 2. Load the tokenizer from the FLAN-T5-large checkpoint
tokenizer = None

# 3. Define the preprocessing function that now also computes decoder inputs and attention masks.
prefix = "translate English to French: "

# Custom implementation of shift_tokens_right
def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right, placing the decoder_start_token_id at the beginning.
    Replace any -100 values with pad_token_id.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 0] = decoder_start_token_id
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    # Replace -100 values with pad_token_id
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)
    return shifted_input_ids

def preprocess_function(examples):
   None

# Map the preprocessing function over the datasets (batched mode)
train_dataset = None
eval_dataset = None


In [ ]:

# 4. Load the FLAN-T5-large model using AutoModel and wrap it with PEFT LoRA
model = None
lora_config = None
model = get_peft_model(model, lora_config)
print("Model wrapped with LoRA configuration.")

# 5. Prepare the data collator (for dynamic padding)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
from transformers import TrainerCallback
# Custom callback to print trainable parameters at training start.
class PrintTrainableParamsCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        model = kwargs.get("model")
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total parameters: {total_params}")
        print(f"Trainable parameters: {trainable_params}")
        return control


In [ ]:
# 6. Set up training arguments
training_args =

# 7. Initialize the Trainer with the custom callback and using processing_class instead of tokenizer.
trainer =


# 8. Fine-tune the model
print("Starting fine-tuning...")
# trainer.train()
print("Fine-tuning complete.")

# Optionally save the fine-tuned model (including LoRA adapters)
model.save_pretrained("./flan_t5_finetuned_lora")
tokenizer.save_pretrained("./flan_t5_finetuned_lora")


In [ ]:

# 9. Define a helper function for translation using the fine-tuned model
def translate_text(text):
    prompt = prefix + text
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=128, num_beams=4)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation.replace(prefix, "").strip()

# 10. Chatbot loop: send translation prompts and get responses
if __name__ == "__main__":
    print("\nTranslation Chatbot (type 'exit' to quit)")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            break
        translation = translate_text(user_input)
        print("Bot (French Translation):", translation)


In [ ]:
# prompt: make a zip file out of the files flan_t5_finetuned_opus_books_lora and download to local computer

!zip -r flan_t5_finetuned_opus_books_lora.zip flan_t5_finetuned_opus_books_lora
!ls

#To download the file, you can use the following code in a code cell:
#from google.colab import files
#files.download('flan_t5_finetuned_opus_books_lora.zip')
